In [1]:
import numpy as np
import torch

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
display(device)

device(type='cuda')

In [5]:
np.random.seed(45)
num_samples = 40
    
# Generate data
x1 = np.random.uniform(-1, 1, num_samples)
f_x = 3*x1 + 4
eps = np.random.randn(num_samples)
y = f_x + eps

In [7]:
display(x1, f_x, y, eps)

array([ 0.97802303,  0.09908945, -0.4371054 , -0.84542087, -0.11106101,
       -0.05438406, -0.902956  , -0.6733511 , -0.76809858,  0.25478337,
        0.7123641 ,  0.30020484,  0.98144337, -0.0592985 ,  0.23658897,
       -0.43466559,  0.95200663,  0.346136  , -0.11893822, -0.42062532,
        0.01939937, -0.77507821, -0.54609042, -0.04289539, -0.51448369,
       -0.22403496,  0.63774686, -0.85091843,  0.84629908, -0.55020784,
        0.41274297, -0.77886909,  0.20200825, -0.18641492,  0.6736589 ,
       -0.49993914, -0.08452402,  0.11486541, -0.4960996 , -0.77951736])

array([6.93406908, 4.29726836, 2.68868381, 1.4637374 , 3.66681698,
       3.83684782, 1.29113201, 1.97994669, 1.69570427, 4.7643501 ,
       6.13709229, 4.90061453, 6.94433011, 3.82210449, 4.7097669 ,
       2.69600324, 6.85601989, 5.03840799, 3.64318533, 2.73812404,
       4.0581981 , 1.67476537, 2.36172874, 3.87131383, 2.45654894,
       3.32789511, 5.91324057, 1.44724471, 6.53889723, 2.34937649,
       5.23822891, 1.66339274, 4.60602474, 3.44075523, 6.0209767 ,
       2.50018257, 3.74642795, 4.34459622, 2.5117012 , 1.66144793])

array([6.0030427 , 5.40825442, 2.49900085, 2.74214699, 3.11273986,
       4.19115056, 0.85085583, 1.55549786, 1.92378589, 5.34339426,
       5.28290394, 4.05927419, 6.288626  , 3.67522179, 3.2977914 ,
       2.42544641, 7.9853232 , 3.69814992, 3.3938893 , 4.50161625,
       3.46623868, 0.76695369, 2.63352835, 3.86675216, 3.31065588,
       1.73789022, 5.96928955, 1.55310728, 6.44805966, 1.63550044,
       5.4419443 , 2.86642458, 4.69033781, 3.60468915, 6.39472825,
       2.27327572, 3.07515354, 4.6587061 , 3.99611931, 2.31361209])

array([-0.93102638,  1.11098606, -0.18968296,  1.27840959, -0.55407712,
        0.35430274, -0.44027617, -0.42444883,  0.22808162,  0.57904417,
       -0.85418835, -0.84134033, -0.65570411, -0.1468827 , -1.4119755 ,
       -0.27055682,  1.1293033 , -1.34025807, -0.24929603,  1.76349221,
       -0.59195943, -0.90781168,  0.27179962, -0.00456167,  0.85410694,
       -1.59000488,  0.05604899,  0.10586257, -0.09083758, -0.71387606,
        0.20371539,  1.20303184,  0.08431307,  0.16393392,  0.37375155,
       -0.22690685, -0.6712744 ,  0.31410988,  1.48441811,  0.65216416])